In [1]:
!pip install alpha_vantage

In [2]:
from alpha_vantage.fundamentaldata import FundamentalData
import json
import pandas as pd
import requests
import time
import bs4 as bs
import datetime as dt
import io
import numpy as np
import time


In [ ]:
apikey = 'GetYourOwn'


In [4]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies') #web page we are looking at
soup = bs.BeautifulSoup(resp.text, 'lxml') # we are going to parse it as lxml
table = soup.find('table', {'class': 'wikitable sortable'}) #find the wiki table on the page

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
    tickers = [s.replace('\n', "") for s in tickers]


In [5]:
tickers.remove('BF.B')
tickers.remove('BRK.B')

In [6]:
fd = FundamentalData(key = apikey, output_format='pandas')


In [7]:
#Take a look at the different variables available to us for the different reports. I will then choose only the variables I feel are relevant to the analysis
ticker = 'AAPL'
overview, o_meta = fd.get_company_overview(symbol = ticker)
income, i_meta = fd.get_income_statement_quarterly(symbol = ticker)
balance, b_meta = fd.get_balance_sheet_quarterly(symbol = ticker)
cashflow, c_meta = fd.get_cash_flow_quarterly(symbol = ticker)

print(overview.columns)
print(income.columns)
print(balance.columns)
print(cashflow.columns)



Index(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange',
       'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd',
       'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio',
       'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS',
       'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM',
       'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice',
       'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold',
       'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE',
       'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue',
       'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage',
       '200DayMovingAverage', 'SharesOutstanding', 'DividendDate',
       'ExDividendDate'],
      dtype='object')
Index(['fiscalDateEnding', '

In [8]:
overview_cols = ['Symbol','MarketCapitalization', 'EBITDA', 'PERatio',
       'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'TrailingPE', 'Beta']
income_cols = ['grossProfit','totalRevenue','costOfRevenue','operatingIncome','operatingExpenses','ebit','netIncome']
balance_cols = ['totalAssets','totalLiabilities','currentDebt','totalShareholderEquity']
cashflow_cols = ['operatingCashflow','capitalExpenditures','profitLoss','dividendPayout']

In [9]:
# The API will return four dataframes for each of the companies. I need to write a function that takes those dataframes and turns them into a sinigle line item in a new dataframe
def add_company(overview,income,balance,cahsflow):
  df = overview[overview_cols].copy()
  df[income_cols] = income[income_cols].iloc[0:1].reset_index(drop=True).copy()
  df[balance_cols] = balance[balance_cols].iloc[0:1].reset_index(drop=True).copy()
  df[cashflow_cols] = cashflow[cashflow_cols].iloc[0:1].reset_index(drop=True).copy()
  return df


In [11]:
dat = pd.DataFrame()
for ticker in tickers:

  overview, o_meta = fd.get_company_overview(symbol = ticker)
  income, i_meta = fd.get_income_statement_annual(symbol = ticker)
  balance, b_meta = fd.get_balance_sheet_annual(symbol = ticker)
  cashflow, c_meta = fd.get_cash_flow_annual(symbol = ticker)

  company = add_company(overview,income,balance,cashflow)
  dat = pd.concat([dat, company], ignore_index=True)
  time.sleep(1)




In [12]:
dat.head()

,Symbol,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,GrossProfitTTM,...,ebit,netIncome,totalAssets,totalLiabilities,currentDebt,totalShareholderEquity,operatingCashflow,capitalExpenditures,profitLoss,dividendPayout
0,MMM,50748916000,7871000000,None,2.051,8.7,6,0.0658,-12.63,15000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,12009640000,819200000,22.11,2.015,12.5,1.22,0.0157,3.69,1329600000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABT,205948518000,10463000000,36.41,28.67,22.26,2.08,0.0185,3.26,24579000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBV,313954697000,26358000000,65.34,1.666,5.87,5.99,0.0347,2.72,41527000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACN,236676940000,11303738000,34.98,2.546,42.55,4.82,0.0137,10.79,20731607000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
dat.shape

(501, 30)

In [14]:
dat.to_csv('fundamentals.csv')